In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Creating function to save predictions to csv

In [2]:
def predict_tocsv(yhat_test):
    predicted = pd.DataFrame({"Id": np.arange(1461,2920), "SalePrice": yhat_test})
    return predicted.to_csv("./predicted.csv", index=False)

### Preparing Training Validation Testing

In [94]:
from sklearn.model_selection import train_test_split

train = pd.read_pickle("../train.pkl")
test = pd.read_pickle("../test.pkl")
temp = pd.read_csv("../data/train.csv")

#importing Saleprice from original dataframe
train["SalePrice"] = temp.SalePrice

#removing outliers
train = train[~((train['GrLivArea'] > 4000) & (train['SalePrice'] < 300000))]
train = train[~((train['MasVnrArea'] > 1400) & (train['SalePrice'] < 300000))]
train = train[~((train['LotArea'] > 200000) & (train['SalePrice'] < 500000))]
train = train[~((train['LotFrontage'] > 300) & (train['SalePrice'] < 300000))]
train = train[~((train['OpenPorchSF'] > 500) & (train['SalePrice'] < 100000))]

#Redefining X and y variables
X = train.drop(["SalePrice"], axis=1)
y = train["SalePrice"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

### Finding best n_estimators

In [4]:
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV 

C:\Users\rayl1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\rayl1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [77]:
#Finding best number of n_estimators for booster="gbtree"
results = []

max_depth = np.arange(3,8)
for depth in max_depth:
    xgb = XGBRegressor(max_depth=depth, n_estimators=10000, random_state=100)
    xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)],
        eval_metric="rmse", early_stopping_rounds=50)
    
    
    results.append({
        "max_depth": depth,
        "best_iter": xgb.best_iteration,
        "lowest_RMSE": xgb.best_score
    })


[0]	validation_0-rmse:173630
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:157281
[2]	validation_0-rmse:142338
[3]	validation_0-rmse:128669
[4]	validation_0-rmse:116761
[5]	validation_0-rmse:106171
[6]	validation_0-rmse:96668
[7]	validation_0-rmse:88056.3
[8]	validation_0-rmse:80345.5
[9]	validation_0-rmse:73584.3
[10]	validation_0-rmse:67559
[11]	validation_0-rmse:62119
[12]	validation_0-rmse:57378.6
[13]	validation_0-rmse:53052.9
[14]	validation_0-rmse:49522.9
[15]	validation_0-rmse:46169.4
[16]	validation_0-rmse:43211.7
[17]	validation_0-rmse:40643.2
[18]	validation_0-rmse:38315.2
[19]	validation_0-rmse:36370.2
[20]	validation_0-rmse:34784
[21]	validation_0-rmse:33347.6
[22]	validation_0-rmse:32048.4
[23]	validation_0-rmse:30879
[24]	validation_0-rmse:29913.6
[25]	validation_0-rmse:29260.4
[26]	validation_0-rmse:28417.4
[27]	validation_0-rmse:27891
[28]	validation_0-rmse:27498.9
[29]	validation_0-rmse:27028
[30]	validation_0-rmse:26701.1
[31]

[260]	validation_0-rmse:21951.9
[261]	validation_0-rmse:21960.1
[262]	validation_0-rmse:21952.6
[263]	validation_0-rmse:21931.1
[264]	validation_0-rmse:21931.1
[265]	validation_0-rmse:21933
[266]	validation_0-rmse:21938.9
[267]	validation_0-rmse:21929.1
[268]	validation_0-rmse:21933.7
[269]	validation_0-rmse:21918.5
[270]	validation_0-rmse:21913
[271]	validation_0-rmse:21906.9
[272]	validation_0-rmse:21913.6
[273]	validation_0-rmse:21915
[274]	validation_0-rmse:21916.2
[275]	validation_0-rmse:21907.1
[276]	validation_0-rmse:21917.6
[277]	validation_0-rmse:21910.7
[278]	validation_0-rmse:21932.9
[279]	validation_0-rmse:21943.5
[280]	validation_0-rmse:21925.1
[281]	validation_0-rmse:21922.4
[282]	validation_0-rmse:21904.5
[283]	validation_0-rmse:21910.8
[284]	validation_0-rmse:21905.7
[285]	validation_0-rmse:21908.2
[286]	validation_0-rmse:21890.7
[287]	validation_0-rmse:21883.2
[288]	validation_0-rmse:21886.6
[289]	validation_0-rmse:21881.1
[290]	validation_0-rmse:21892.9
[291]	validati

[518]	validation_0-rmse:21499.5
[519]	validation_0-rmse:21500.2
[520]	validation_0-rmse:21505.5
[521]	validation_0-rmse:21500.2
[522]	validation_0-rmse:21493.4
[523]	validation_0-rmse:21494.8
[524]	validation_0-rmse:21498.2
[525]	validation_0-rmse:21498.5
[526]	validation_0-rmse:21494.4
[527]	validation_0-rmse:21490.9
[528]	validation_0-rmse:21494.7
[529]	validation_0-rmse:21495.5
[530]	validation_0-rmse:21481.8
[531]	validation_0-rmse:21487
[532]	validation_0-rmse:21481.3
[533]	validation_0-rmse:21483.7
[534]	validation_0-rmse:21492.8
[535]	validation_0-rmse:21492.6
[536]	validation_0-rmse:21494.9
[537]	validation_0-rmse:21500.6
[538]	validation_0-rmse:21499.6
[539]	validation_0-rmse:21495
[540]	validation_0-rmse:21492.2
[541]	validation_0-rmse:21489.9
[542]	validation_0-rmse:21482.8
[543]	validation_0-rmse:21485.7
[544]	validation_0-rmse:21475.8
[545]	validation_0-rmse:21468.2
[546]	validation_0-rmse:21471
[547]	validation_0-rmse:21462.3
[548]	validation_0-rmse:21466.1
[549]	validati

[117]	validation_0-rmse:24147.6
[118]	validation_0-rmse:24162
[119]	validation_0-rmse:24143.5
[120]	validation_0-rmse:24124.3
[121]	validation_0-rmse:24141.5
[122]	validation_0-rmse:24146.4
[123]	validation_0-rmse:24135.9
[124]	validation_0-rmse:24135.5
[125]	validation_0-rmse:24122.9
[126]	validation_0-rmse:24116.6
[127]	validation_0-rmse:24120.3
[128]	validation_0-rmse:24126.9
[129]	validation_0-rmse:24136.2
[130]	validation_0-rmse:24129.7
[131]	validation_0-rmse:24125.2
[132]	validation_0-rmse:24121.7
[133]	validation_0-rmse:24105.5
[134]	validation_0-rmse:24079.3
[135]	validation_0-rmse:24096.6
[136]	validation_0-rmse:24106.3
[137]	validation_0-rmse:24086.8
[138]	validation_0-rmse:24074.7
[139]	validation_0-rmse:24079.7
[140]	validation_0-rmse:24072.2
[141]	validation_0-rmse:24067.4
[142]	validation_0-rmse:24055.8
[143]	validation_0-rmse:24070.5
[144]	validation_0-rmse:24072.3
[145]	validation_0-rmse:24068.6
[146]	validation_0-rmse:24058.8
[147]	validation_0-rmse:24039.6
[148]	vali

[160]	validation_0-rmse:25076.7
[161]	validation_0-rmse:25073.8
[162]	validation_0-rmse:25062
[163]	validation_0-rmse:25052.5
[164]	validation_0-rmse:25048.6
[165]	validation_0-rmse:25054
[166]	validation_0-rmse:25053.8
[167]	validation_0-rmse:25053
[168]	validation_0-rmse:25056.8
[169]	validation_0-rmse:25057
[170]	validation_0-rmse:25061.8
[171]	validation_0-rmse:25052.1
[172]	validation_0-rmse:25052.3
[173]	validation_0-rmse:25053.3
[174]	validation_0-rmse:25058.2
[175]	validation_0-rmse:25063.2
[176]	validation_0-rmse:25060.8
[177]	validation_0-rmse:25057.1
[178]	validation_0-rmse:25052.6
[179]	validation_0-rmse:25051.9
[180]	validation_0-rmse:25059.4
[181]	validation_0-rmse:25060.2
[182]	validation_0-rmse:25064.1
[183]	validation_0-rmse:25060.7
[184]	validation_0-rmse:25061.8
[185]	validation_0-rmse:25070.7
[186]	validation_0-rmse:25062.9
[187]	validation_0-rmse:25066.7
[188]	validation_0-rmse:25051.5
[189]	validation_0-rmse:25048.8
[190]	validation_0-rmse:25050.9
[191]	validation

[143]	validation_0-rmse:24269
[144]	validation_0-rmse:24272
[145]	validation_0-rmse:24277.6
[146]	validation_0-rmse:24277.3
[147]	validation_0-rmse:24275.3
[148]	validation_0-rmse:24270.5
[149]	validation_0-rmse:24274.6
[150]	validation_0-rmse:24278.3
[151]	validation_0-rmse:24286.4
[152]	validation_0-rmse:24277.2
[153]	validation_0-rmse:24279.3
[154]	validation_0-rmse:24274.7
[155]	validation_0-rmse:24259.8
[156]	validation_0-rmse:24257.5
[157]	validation_0-rmse:24257.2
[158]	validation_0-rmse:24256.9
[159]	validation_0-rmse:24250.4
[160]	validation_0-rmse:24256.1
[161]	validation_0-rmse:24254.2
[162]	validation_0-rmse:24252.5
[163]	validation_0-rmse:24240.6
[164]	validation_0-rmse:24245
[165]	validation_0-rmse:24247.4
[166]	validation_0-rmse:24252.3
[167]	validation_0-rmse:24247.6
[168]	validation_0-rmse:24249.1
[169]	validation_0-rmse:24247.5
[170]	validation_0-rmse:24249.4
[171]	validation_0-rmse:24249.5
[172]	validation_0-rmse:24252.6
[173]	validation_0-rmse:24254.8
[174]	validati

In [78]:
results

[{'max_depth': 3, 'best_iter': 608, 'lowest_RMSE': 21417.851563},
 {'max_depth': 4, 'best_iter': 163, 'lowest_RMSE': 23936.080078},
 {'max_depth': 5, 'best_iter': 224, 'lowest_RMSE': 24967.382813},
 {'max_depth': 6, 'best_iter': 163, 'lowest_RMSE': 24240.615234},
 {'max_depth': 7, 'best_iter': 125, 'lowest_RMSE': 25309.503906}]

In [6]:
#Finding best number of n_estimators for booster="gblinear" - doesnt early stop
results = []

max_depth = np.arange(3,8)
for depth in max_depth:
    xgb = XGBRegressor(booster = "gblinear", learning_rate = 1, max_depth=depth, n_estimators=1000)
    xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric="rmse", early_stopping_rounds=2)
    
    results.append({
        "max_depth": depth,
        "best_iter": xgb.best_iteration,
        "lowest_RMSE": xgb.best_score
    })

[0]	validation_0-rmse:48190
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:45785.8
[2]	validation_0-rmse:45085.3
[3]	validation_0-rmse:44206.7
[4]	validation_0-rmse:43402.4
[5]	validation_0-rmse:42712.3
[6]	validation_0-rmse:42106
[7]	validation_0-rmse:41563
[8]	validation_0-rmse:41074.5
[9]	validation_0-rmse:40635.2
[10]	validation_0-rmse:40239.8
[11]	validation_0-rmse:39883.4
[12]	validation_0-rmse:39561.5
[13]	validation_0-rmse:39270
[14]	validation_0-rmse:39005.6
[15]	validation_0-rmse:38765.4
[16]	validation_0-rmse:38547
[17]	validation_0-rmse:38348.1
[18]	validation_0-rmse:38166.7
[19]	validation_0-rmse:38001.3
[20]	validation_0-rmse:37850.1
[21]	validation_0-rmse:37711.9
[22]	validation_0-rmse:37585.4
[23]	validation_0-rmse:37469.5
[24]	validation_0-rmse:37363.2
[25]	validation_0-rmse:37265.6
[26]	validation_0-rmse:37176
[27]	validation_0-rmse:37093.4
[28]	validation_0-rmse:37017.3
[29]	validation_0-rmse:36947.1
[30]	validation_0-rmse:36882

[259]	validation_0-rmse:35056.6
[260]	validation_0-rmse:35056
[261]	validation_0-rmse:35055.3
[262]	validation_0-rmse:35054.7
[263]	validation_0-rmse:35054.1
[264]	validation_0-rmse:35053.4
[265]	validation_0-rmse:35052.8
[266]	validation_0-rmse:35052.2
[267]	validation_0-rmse:35051.6
[268]	validation_0-rmse:35051
[269]	validation_0-rmse:35050.4
[270]	validation_0-rmse:35049.8
[271]	validation_0-rmse:35049.2
[272]	validation_0-rmse:35048.7
[273]	validation_0-rmse:35048.1
[274]	validation_0-rmse:35047.5
[275]	validation_0-rmse:35047
[276]	validation_0-rmse:35046.4
[277]	validation_0-rmse:35045.9
[278]	validation_0-rmse:35045.4
[279]	validation_0-rmse:35044.8
[280]	validation_0-rmse:35044.3
[281]	validation_0-rmse:35043.8
[282]	validation_0-rmse:35043.2
[283]	validation_0-rmse:35042.7
[284]	validation_0-rmse:35042.2
[285]	validation_0-rmse:35041.7
[286]	validation_0-rmse:35041.2
[287]	validation_0-rmse:35040.7
[288]	validation_0-rmse:35040.2
[289]	validation_0-rmse:35039.8
[290]	validati

[517]	validation_0-rmse:34988.3
[518]	validation_0-rmse:34988.2
[519]	validation_0-rmse:34988.1
[520]	validation_0-rmse:34987.9
[521]	validation_0-rmse:34987.8
[522]	validation_0-rmse:34987.7
[523]	validation_0-rmse:34987.6
[524]	validation_0-rmse:34987.5
[525]	validation_0-rmse:34987.4
[526]	validation_0-rmse:34987.3
[527]	validation_0-rmse:34987.2
[528]	validation_0-rmse:34987.1
[529]	validation_0-rmse:34987
[530]	validation_0-rmse:34986.9
[531]	validation_0-rmse:34986.8
[532]	validation_0-rmse:34986.6
[533]	validation_0-rmse:34986.5
[534]	validation_0-rmse:34986.4
[535]	validation_0-rmse:34986.3
[536]	validation_0-rmse:34986.2
[537]	validation_0-rmse:34986.1
[538]	validation_0-rmse:34986
[539]	validation_0-rmse:34985.9
[540]	validation_0-rmse:34985.8
[541]	validation_0-rmse:34985.7
[542]	validation_0-rmse:34985.6
[543]	validation_0-rmse:34985.5
[544]	validation_0-rmse:34985.4
[545]	validation_0-rmse:34985.3
[546]	validation_0-rmse:34985.2
[547]	validation_0-rmse:34985.1
[548]	valida

[775]	validation_0-rmse:34966.3
[776]	validation_0-rmse:34966.3
[777]	validation_0-rmse:34966.2
[778]	validation_0-rmse:34966.1
[779]	validation_0-rmse:34966.1
[780]	validation_0-rmse:34966
[781]	validation_0-rmse:34965.9
[782]	validation_0-rmse:34965.8
[783]	validation_0-rmse:34965.8
[784]	validation_0-rmse:34965.7
[785]	validation_0-rmse:34965.6
[786]	validation_0-rmse:34965.6
[787]	validation_0-rmse:34965.5
[788]	validation_0-rmse:34965.4
[789]	validation_0-rmse:34965.3
[790]	validation_0-rmse:34965.3
[791]	validation_0-rmse:34965.2
[792]	validation_0-rmse:34965.1
[793]	validation_0-rmse:34965
[794]	validation_0-rmse:34965
[795]	validation_0-rmse:34964.9
[796]	validation_0-rmse:34964.8
[797]	validation_0-rmse:34964.8
[798]	validation_0-rmse:34964.7
[799]	validation_0-rmse:34964.6
[800]	validation_0-rmse:34964.6
[801]	validation_0-rmse:34964.5
[802]	validation_0-rmse:34964.4
[803]	validation_0-rmse:34964.3
[804]	validation_0-rmse:34964.3
[805]	validation_0-rmse:34964.2
[806]	validati

[33]	validation_0-rmse:36714.3
[34]	validation_0-rmse:36666
[35]	validation_0-rmse:36620.8
[36]	validation_0-rmse:36578.4
[37]	validation_0-rmse:36538.7
[38]	validation_0-rmse:36501.2
[39]	validation_0-rmse:36465.8
[40]	validation_0-rmse:36432.3
[41]	validation_0-rmse:36400.5
[42]	validation_0-rmse:36370.2
[43]	validation_0-rmse:36341.2
[44]	validation_0-rmse:36313.4
[45]	validation_0-rmse:36286.7
[46]	validation_0-rmse:36261
[47]	validation_0-rmse:36236.2
[48]	validation_0-rmse:36212.2
[49]	validation_0-rmse:36188.9
[50]	validation_0-rmse:36166.3
[51]	validation_0-rmse:36144.3
[52]	validation_0-rmse:36122.8
[53]	validation_0-rmse:36101.8
[54]	validation_0-rmse:36081.3
[55]	validation_0-rmse:36061.2
[56]	validation_0-rmse:36041.5
[57]	validation_0-rmse:36022.2
[58]	validation_0-rmse:36003.3
[59]	validation_0-rmse:35984.7
[60]	validation_0-rmse:35966.4
[61]	validation_0-rmse:35948.5
[62]	validation_0-rmse:35930.8
[63]	validation_0-rmse:35913.5
[64]	validation_0-rmse:35896.4
[65]	validat

...

[877]	validation_0-rmse:34959.2
[878]	validation_0-rmse:34959.1
[879]	validation_0-rmse:34959
[880]	validation_0-rmse:34959
[881]	validation_0-rmse:34958.9
[882]	validation_0-rmse:34958.8
[883]	validation_0-rmse:34958.8
[884]	validation_0-rmse:34958.7
[885]	validation_0-rmse:34958.6
[886]	validation_0-rmse:34958.5
[887]	validation_0-rmse:34958.5
[888]	validation_0-rmse:34958.4
[889]	validation_0-rmse:34958.3
[890]	validation_0-rmse:34958.3
[891]	validation_0-rmse:34958.2
[892]	validation_0-rmse:34958.1
[893]	validation_0-rmse:34958.1
[894]	validation_0-rmse:34958
[895]	validation_0-rmse:34957.9
[896]	validation_0-rmse:34957.9
[897]	validation_0-rmse:34957.8
[898]	validation_0-rmse:34957.7
[899]	validation_0-rmse:34957.7
[900]	validation_0-rmse:34957.6
[901]	validation_0-rmse:34957.5
[902]	validation_0-rmse:34957.5
[903]	validation_0-rmse:34957.4
[904]	validation_0-rmse:34957.3
[905]	validation_0-rmse:34957.2
[906]	validation_0-rmse:34957.2
[907]	validation_0-rmse:34957.1
[908]	validati

[137]	validation_0-rmse:35248.2
[138]	validation_0-rmse:35244.6
[139]	validation_0-rmse:35241.1
[140]	validation_0-rmse:35237.8
[141]	validation_0-rmse:35234.4
[142]	validation_0-rmse:35231.2
[143]	validation_0-rmse:35228
[144]	validation_0-rmse:35224.8
[145]	validation_0-rmse:35221.8
[146]	validation_0-rmse:35218.7
[147]	validation_0-rmse:35215.8
[148]	validation_0-rmse:35212.9
[149]	validation_0-rmse:35210
[150]	validation_0-rmse:35207.2
[151]	validation_0-rmse:35204.4
[152]	validation_0-rmse:35201.7
[153]	validation_0-rmse:35199.1
[154]	validation_0-rmse:35196.5
[155]	validation_0-rmse:35193.9
[156]	validation_0-rmse:35191.4
[157]	validation_0-rmse:35188.9
[158]	validation_0-rmse:35186.5
[159]	validation_0-rmse:35184.1
[160]	validation_0-rmse:35181.7
[161]	validation_0-rmse:35179.4
[162]	validation_0-rmse:35177.1
[163]	validation_0-rmse:35174.9
[164]	validation_0-rmse:35172.7
[165]	validation_0-rmse:35170.5
[166]	validation_0-rmse:35168.4
[167]	validation_0-rmse:35166.3
[168]	valida

[395]	validation_0-rmse:35006.4
[396]	validation_0-rmse:35006.2
[397]	validation_0-rmse:35006
[398]	validation_0-rmse:35005.8
[399]	validation_0-rmse:35005.6
[400]	validation_0-rmse:35005.4
[401]	validation_0-rmse:35005.2
[402]	validation_0-rmse:35005
[403]	validation_0-rmse:35004.8
[404]	validation_0-rmse:35004.6
[405]	validation_0-rmse:35004.4
[406]	validation_0-rmse:35004.2
[407]	validation_0-rmse:35004
[408]	validation_0-rmse:35003.9
[409]	validation_0-rmse:35003.7
[410]	validation_0-rmse:35003.5
[411]	validation_0-rmse:35003.3
[412]	validation_0-rmse:35003.1
[413]	validation_0-rmse:35002.9
[414]	validation_0-rmse:35002.8
[415]	validation_0-rmse:35002.6
[416]	validation_0-rmse:35002.4
[417]	validation_0-rmse:35002.2
[418]	validation_0-rmse:35002
[419]	validation_0-rmse:35001.9
[420]	validation_0-rmse:35001.7
[421]	validation_0-rmse:35001.5
[422]	validation_0-rmse:35001.3
[423]	validation_0-rmse:35001.2
[424]	validation_0-rmse:35001
[425]	validation_0-rmse:35000.8
[426]	validation_0

[653]	validation_0-rmse:34975.6
[654]	validation_0-rmse:34975.5
[655]	validation_0-rmse:34975.5
[656]	validation_0-rmse:34975.4
[657]	validation_0-rmse:34975.3
[658]	validation_0-rmse:34975.2
[659]	validation_0-rmse:34975.1
[660]	validation_0-rmse:34975
[661]	validation_0-rmse:34975
[662]	validation_0-rmse:34974.9
[663]	validation_0-rmse:34974.8
[664]	validation_0-rmse:34974.7
[665]	validation_0-rmse:34974.6
[666]	validation_0-rmse:34974.6
[667]	validation_0-rmse:34974.5
[668]	validation_0-rmse:34974.4
[669]	validation_0-rmse:34974.3
[670]	validation_0-rmse:34974.3
[671]	validation_0-rmse:34974.2
[672]	validation_0-rmse:34974.1
[673]	validation_0-rmse:34974
[674]	validation_0-rmse:34973.9
[675]	validation_0-rmse:34973.8
[676]	validation_0-rmse:34973.8
[677]	validation_0-rmse:34973.7
[678]	validation_0-rmse:34973.6
[679]	validation_0-rmse:34973.5
[680]	validation_0-rmse:34973.5
[681]	validation_0-rmse:34973.4
[682]	validation_0-rmse:34973.3
[683]	validation_0-rmse:34973.2
[684]	validati

[911]	validation_0-rmse:34956.8
[912]	validation_0-rmse:34956.8
[913]	validation_0-rmse:34956.7
[914]	validation_0-rmse:34956.6
[915]	validation_0-rmse:34956.6
[916]	validation_0-rmse:34956.5
[917]	validation_0-rmse:34956.4
[918]	validation_0-rmse:34956.4
[919]	validation_0-rmse:34956.3
[920]	validation_0-rmse:34956.2
[921]	validation_0-rmse:34956.2
[922]	validation_0-rmse:34956.1
[923]	validation_0-rmse:34956
[924]	validation_0-rmse:34955.9
[925]	validation_0-rmse:34955.9
[926]	validation_0-rmse:34955.8
[927]	validation_0-rmse:34955.8
[928]	validation_0-rmse:34955.7
[929]	validation_0-rmse:34955.6
[930]	validation_0-rmse:34955.6
[931]	validation_0-rmse:34955.5
[932]	validation_0-rmse:34955.4
[933]	validation_0-rmse:34955.4
[934]	validation_0-rmse:34955.3
[935]	validation_0-rmse:34955.2
[936]	validation_0-rmse:34955.1
[937]	validation_0-rmse:34955.1
[938]	validation_0-rmse:34955
[939]	validation_0-rmse:34954.9
[940]	validation_0-rmse:34954.9
[941]	validation_0-rmse:34954.8
[942]	valida

In [7]:
results

[{'max_depth': 3, 'best_iter': 999, 'lowest_RMSE': 34950.910156},
 {'max_depth': 4, 'best_iter': 999, 'lowest_RMSE': 34950.910156},
 {'max_depth': 5, 'best_iter': 999, 'lowest_RMSE': 34950.910156},
 {'max_depth': 6, 'best_iter': 999, 'lowest_RMSE': 34950.910156},
 {'max_depth': 7, 'best_iter': 999, 'lowest_RMSE': 34950.910156}]

### Tuning min_child_weight

In [47]:
#default booster is "gbtree"

param_1 = {
    'min_child_weight': np.arange(1,6,2),
}

grid = GridSearchCV(estimator=XGBRegressor(n_estimators=608, max_depth=3), param_grid=param_1, 
                    scoring="neg_mean_squared_error",  cv=5)
grid.fit(X, y)

grid.best_params_, grid.best_score_

({'max_depth': 3, 'min_child_weight': 3}, -541809522.7378676)

In [54]:
pred_Sales = grid.predict(test)

predict_tocsv(pred_Sales)

In [55]:
#changing booster to "gblinear" - poor scores on kaggle

param_1a = {
    'min_child_weight': np.arange(1,6,2),
}

grid = GridSearchCV(estimator=XGBRegressor(booster="gblinear", n_estimators=350, max_depth=3), param_grid=param_1a, 
                    scoring="neg_mean_squared_error", n_jobs=-1, cv=5)
grid.fit(X, y)

grid.best_params_, grid.best_score_

({'min_child_weight': 1}, -1231103649.5317636)

### Tuning subsample and lambda

In [15]:
from sklearn.metrics import r2_score

param_2 = {
    'subsample':[i/100.0 for i in range(75,90,5)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

grid = GridSearchCV(estimator=XGBRegressor(n_estimators=134, max_depth=3), param_grid=param_1, 
                    scoring="neg_mean_squared_error", cv=5)
grid.fit(X, y)

grid.best_params_, grid.best_score_

({'reg_alpha': 1e-05, 'subsample': 0.85}, -528860030.35631573)

In [16]:
pred_Sales = grid.predict(test)

predict_tocsv(pred_Sales)

### Random Forest

In [19]:
from sklearn.ensemble import RandomForestRegressor

rf_param = {
    "n_estimators": np.linspace(start=100, stop=1000, num=10, dtype=int),
    "min_samples_leaf": np.linspace(2,50, 5, dtype=int),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "random_state": [100]}

rf = RandomForestRegressor(oob_score=True)
grid_rf = GridSearchCV(rf, rf_param, cv=5, n_jobs=-1)


C:\Users\rayl1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [20]:
grid_rf.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000]), 'min_samples_leaf': array([ 2, 14, 26, 38, 50]), 'min_samples_split': array([ 2,  9, 16, 23, 30]), 'random_state': [100]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [21]:
grid_rf.best_params_, grid_rf.best_score_

({'min_samples_leaf': 2,
  'min_samples_split': 2,
  'n_estimators': 600,
  'random_state': 100},
 0.8871448121236052)

In [22]:
pred_Sales = grid_rf.predict(test)

predict_tocsv(pred_Sales)

### Creating Noise Variable and Feature Importance

In [6]:
train_rn = train.copy()
train_rn["Random_Noise"] = np.random.random(size=len(train_rn))

X_rn = train_rn.drop("SalePrice", axis=1)
y_rn = train_rn.SalePrice

rf = RandomForestRegressor(n_estimators = 600, random_state=100, max_features=20, oob_score=True)
rf.fit(X_rn, y_rn)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=20, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=600, n_jobs=1, oob_score=True, random_state=100,
           verbose=0, warm_start=False)

In [8]:
pd.options.display.max_rows = 300

importance = pd.DataFrame({"values":rf.feature_importances_}, index=X_rn.columns).sort_values("values",ascending=False)
importance = importance.reset_index()
importance

,index,values
0,OverallQual,8.924844e-02
1,GrLivArea,7.059505e-02
2,TotalBsmtSF,6.238776e-02
3,GarageCars,5.563114e-02
4,YearBuilt,4.003305e-02
5,ExterQual_TA,3.776248e-02
6,BsmtQual_Ex,3.605717e-02
7,BsmtFinSF1,2.987842e-02
8,GarageYrBlt,2.826024e-02
9,LotArea,2.714857e-02


### Fitting Random Forest using variable importance

In [22]:
from sklearn.ensemble import RandomForestRegressor

#instantiating parameters for gridsearch on RF
rf_param = {
    "n_estimators": np.linspace(start=650, stop=750, num=10, dtype=int),
    "random_state": [10,30,50,70,90]}

rf_rnbest = RandomForestRegressor(oob_score=True)
grid_rf_rnbest = GridSearchCV(rf_rnbest, rf_param, cv=5, n_jobs=-1)


In [23]:
#Fitting gridsearch based on important features
rf_important_features = list(importance["index"].iloc[0:importance.index[importance["index"]=="Random_Noise"][0]])
X_rn_best = X_rn[rf_important_features]

grid_rf_rnbest.fit(X_rn_best,y_rn)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': array([650, 661, 672, 683, 694, 705, 716, 727, 738, 750]), 'random_state': [10, 30, 50, 70, 90]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [24]:
#best parameters
grid_rf_rnbest.best_params_, grid_rf_rnbest.best_score_

({'n_estimators': 661, 'random_state': 30}, 0.8871666222714908)

In [30]:
# subsetting important columns for test dataset and then predicting on those columns
test_rf_rnbest = test[rf_important_features]

pred_Saleprice = grid_rf_rnbest.predict(test_rf_rnbest)

predict_tocsv(pred_Saleprice)

### Lasso

Base Case

In [47]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
#from sklearn.preprocessing import StandardScaler

#scale = StandardScaler()
#scale.fit(X)
#X_std = scale.transform(X)
y_log = np.log1p(y)

lasso = Lasso(alpha = 0.0004, max_iter=1000)

lasso.fit(X, y_log)

Lasso(alpha=0.0004, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [50]:
lasso_prediction = np.expm1(lasso.predict(test))

predict_tocsv(lasso_prediction)

Different Parameters Test for Lasso

In [73]:
#Trying on subsetted data from random forest random noise feature selection

from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

y_log = np.log1p(y)

lasso = LassoCV(alphas = [i/100000 for i in range(2,100)], max_iter=1000, cv=5)

lasso.fit(X_rn_best, y_log)

LassoCV(alphas=[2e-05, 3e-05, 4e-05, 5e-05, 6e-05, 7e-05, 8e-05, 9e-05, 0.0001, 0.00011, 0.00012, 0.00013, 0.00014, 0.00015, 0.00016, 0.00017, 0.00018, 0.00019, 0.0002, 0.00021, 0.00022, 0.00023, 0.00024, 0.00025, 0.00026, 0.00027, 0.00028, 0.00029, 0.0003, 0.00031, 0.00032, 0.00033, 0.00034, 0.00035, 0.000...8, 0.00089, 0.0009, 0.00091, 0.00092, 0.00093, 0.00094, 0.00095, 0.00096, 0.00097, 0.00098, 0.00099],
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [74]:
lasso.score(X_rn_best, y_log)

0.8962030770122816

In [76]:
# subsetting important columns for test dataset and then predicting on those columns
test_rf_rnbest = test[rf_important_features]

pred_Saleprice = np.expm1(lasso.predict(test_rf_rnbest))

predict_tocsv(pred_Saleprice)

### Ridge 

In [86]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

y_log = np.log1p(y)

ridge = RidgeCV(alphas = [i/100000 for i in range(2,100)], cv=5)

ridge.fit(X, y_log)

RidgeCV(alphas=[2e-05, 3e-05, 4e-05, 5e-05, 6e-05, 7e-05, 8e-05, 9e-05, 0.0001, 0.00011, 0.00012, 0.00013, 0.00014, 0.00015, 0.00016, 0.00017, 0.00018, 0.00019, 0.0002, 0.00021, 0.00022, 0.00023, 0.00024, 0.00025, 0.00026, 0.00027, 0.00028, 0.00029, 0.0003, 0.00031, 0.00032, 0.00033, 0.00034, 0.00035, 0.000...8, 0.00089, 0.0009, 0.00091, 0.00092, 0.00093, 0.00094, 0.00095, 0.00096, 0.00097, 0.00098, 0.00099],
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [92]:
predict_ridge = np.expm1(ridge.predict(test))

predict_tocsv(predict_ridge)

In [57]:
from patsy import dmatrices
features = "+".join(X.columns)

y, X = dmatrices('SalePrice ~' + features, train, return_type='dataframe')

SyntaxError: invalid syntax (<unknown>, line 1)

In [51]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
vif["features"] = X.columns

TypeError: unhashable type: 'slice'

### Random Forest with subsetted columns from noise column

In [79]:
#fitting on best selected features from RF model with noise column

param_1 = {
    'min_child_weight': np.arange(1,6,2),
    'subsample':[i/100.0 for i in range(75,90,5)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

grid = GridSearchCV(estimator=XGBRegressor(n_estimators=608, max_depth=3), param_grid=param_1, 
                    scoring="neg_mean_squared_error", n_jobs=-1, cv=5)

grid.fit(X_rn_best, y_log)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=608,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_child_weight': array([1, 3, 5]), 'subsample': [0.75, 0.8, 0.85], 'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

In [80]:
grid.best_params_, grid.best_score_

({'min_child_weight': 3, 'reg_alpha': 1, 'subsample': 0.85},
 -0.01855306577174669)

In [82]:
predicted = np.expm1(grid.predict(test_rf_rnbest))

predict_tocsv(predicted)